In [1]:
import requests
import pandas as pd
from urllib.parse import unquote
import plotly.express as px

# URLs and headers
geturl = 'https://www.barchart.com/futures/quotes/CLF25/options/jan-25?futuresOptionsView=merged'
apiurl = 'https://www.barchart.com/proxies/core-api/v1/quotes/get'

getheaders = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36'
}

getpay = {
    'page': 'all'
}

s = requests.Session()
r = s.get(geturl, params=getpay, headers=getheaders)

headers = {
    'accept': 'application/json',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'referer': 'https://www.barchart.com/futures/quotes/CLF25/options/oct-24?futuresOptionsView=merged',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36',
    'x-xsrf-token': unquote(unquote(s.cookies.get_dict()['XSRF-TOKEN']))
}

payload = {
    'fields': 'symbol,contractSymbol,lastPrice,priceChange,openPrice,highPrice,lowPrice,previousPrice,volume,openInterest,tradeTime,symbolCode,symbolType,hasOptions',
    'list': 'futures.contractInRoot',
    'root': 'CL',
    'meta': 'field.shortName,field.type,field.description',
    'hasOptions': 'true',
    'raw': '1'
}

r = s.get(apiurl, params=payload, headers=headers)
j = r.json()

# Convert JSON to DataFrame
df = pd.json_normalize(j['data'])  # Adjust the key based on the actual structure of your JSON
df = df.drop(0).reset_index(drop=True)
# Print DataFrame
print(df)
first_10_symbols = df['symbol'].iloc[:10].tolist()
print(first_10_symbols)


# Clean the 'lastPrice' column: remove non-numeric characters and convert to float
df['lastPrice'] = df['lastPrice'].str.replace('s', '', regex=False).astype(float)

# Plot using Plotly
fig = px.scatter(df, x='symbol', y='lastPrice', 
                 title='Forward Curve',
                 labels={'symbol': 'Contract Symbol', 'lastPrice': 'Last Price'},)

# Show the plot
fig.show()

    symbol   contractSymbol lastPrice priceChange openPrice highPrice  \
0    CLH25  CLH25 (Mar '25)     71.25       +0.22     71.18     71.31   
1    CLJ25  CLJ25 (Apr '25)     70.92       +0.18     70.88     70.97   
2    CLK25  CLK25 (May '25)     70.47       +0.13     70.37     70.51   
3    CLM25  CLM25 (Jun '25)     69.99       +0.10     69.85     70.03   
4    CLN25  CLN25 (Jul '25)     69.54       +0.12     69.38     69.54   
..     ...              ...       ...         ...       ...       ...   
127  CLV35  CLV35 (Oct '35)    60.66s       +0.27      0.00     60.66   
128  CLX35  CLX35 (Nov '35)    60.64s       +0.27      0.00     60.64   
129  CLZ35  CLZ35 (Dec '35)    60.61s       +0.27      0.00     60.61   
130  CLF36  CLF36 (Jan '36)    60.54s       +0.27      0.00     60.54   
131  CLG36  CLG36 (Feb '36)    60.48s       +0.27      0.00     60.48   

    lowPrice previousPrice volume openInterest  ... raw.openPrice  \
0      71.04         71.03  1,453      276,301  ...   

In [2]:
import requests
import pandas as pd
import sqlite3
from bs4 import BeautifulSoup
import datetime


# Base URL format
base_url = "https://www.barchart.com/futures/quotes/{}/options/"

# Headers to avoid 403 errors
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Referer': 'https://www.google.com/',
    'Accept-Language': 'en-US,en;q=0.9',
}

# List to store extracted data
data = []

# Scrape data for each contract
for contract in first_10_symbols:
    url = base_url.format(contract)
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        toolbar = soup.find('div', class_='row bc-options-toolbar__second-row')

        if toolbar:
            columns = toolbar.find_all('div', class_='column')
        

            days_to_expiry = columns[0].get_text(strip=True)
            implied_volatility = columns[1].get_text(strip=True)

            # Extract only numerical values
            expiration_date = days_to_expiry.split()[3][2:]
            days_to_expiry = int(days_to_expiry.split()[0])
            
            implied_volatility = float(implied_volatility.split(":")[1].replace("%", "").strip())

            # Get last price from df
            last_price = df.loc[df["symbol"] == contract, "lastPrice"].values[0]

            # Append data
            data.append([contract, last_price,expiration_date, days_to_expiry, implied_volatility])
        else:
            print(f"Could not find options toolbar for {contract}.")
    else:
        print(f"Failed to fetch data for {contract}. Status code: {response.status_code}")

# Create DataFrame
futures_df = pd.DataFrame(data, columns=["Contract", "Last Price","Expiration Date", "Options Days to Expiry", " Futures Implied Volatility"])


# Display the DataFrame
print(futures_df)


  Contract  Last Price Expiration Date  Options Days to Expiry  \
0    CLH25       71.25        02/14/25                       9   
1    CLJ25       70.92        03/17/25                      40   
2    CLK25       70.47        04/16/25                      70   
3    CLM25       69.99        05/15/25                      99   
4    CLN25       69.54        06/16/25                     131   
5    CLQ25       69.02        07/17/25                     162   
6    CLU25       68.40        08/15/25                     191   
7    CLV25       68.00        09/17/25                     224   
8    CLX25       67.60        10/16/25                     253   
9    CLZ25       67.29        11/17/25                     285   

    Futures Implied Volatility  
0                        30.00  
1                        28.30  
2                        28.22  
3                        27.76  
4                        27.54  
5                        27.32  
6                        27.19  
7        

In [3]:
import requests
from urllib.parse import unquote
import pandas as pd

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0",
}

def main(url, symbols):
    with requests.Session() as req:
        req.headers.update(headers)
        r = req.get(url[:25])
        req.headers.update(
            {'X-XSRF-TOKEN': unquote(r.cookies.get_dict()['XSRF-TOKEN'])})
        

        results = {}
        for i, symbol in enumerate(symbols, start=1):
            params = {
                "symbol": symbol,
                "list": "futures.options",
                "fields": "strike,openPrice,highPrice,lowPrice,lastPrice,priceChange,bidPrice,askPrice,volume,openInterest,premium,tradeTime,longSymbol,optionType,symbol",
                "orderBy": "strike",
                "orderDir": "asc",
                "meta": "field.shortName,field.description,field.type",
                "futureOptions": "true",
                "noPagination": "true",
                "showExpandLink": "false"
            }
            
            r = req.get(url, params=params).json()
            
            df = pd.DataFrame(r['data'])
            df['strike'] = df['strike'].astype(str).str.replace(r'[CP]', '', regex=True)
            df['volume'] = pd.to_numeric(df['volume'], errors='coerce').fillna(0).astype(int)
            df['openInterest'] = pd.to_numeric(df['openInterest'], errors='coerce').fillna(0).astype(int)
            df['lastPrice'] = df['lastPrice'].str.replace(r'[a-zA-Z]', '', regex=True).astype(float)

            
            
            calls_df = df[df['optionType'] == 'Call']
            puts_df = df[df['optionType'] == 'Put']
            
            results[symbol] = (calls_df, puts_df)

        return results
                

# Specify the URL and the output file path
url = 'https://www.barchart.com/proxies/core-api/v1/quotes/get?'



# Call the main function with the list of symbols and output file
results = main(url, first_10_symbols)



In [4]:
symbols = results.keys()
print(list(symbols))

for symbol, (calls_df, puts_df) in results.items():
    print(f"Symbol: {symbol}")
    
    print("DataFrame Columns:")
    print(calls_df.columns)  # Prints column names of calls DataFrame
    
    print("Calls DataFrame Sample:")
    print(calls_df.head())  # Prints first five rows of calls DataFrame
    
    print("Puts DataFrame Sample:")
    print(puts_df.head())  # Prints first five rows of puts DataFrame


['CLH25', 'CLJ25', 'CLK25', 'CLM25', 'CLN25', 'CLQ25', 'CLU25', 'CLV25', 'CLX25', 'CLZ25']
Symbol: CLH25
DataFrame Columns:
Index(['strike', 'openPrice', 'highPrice', 'lowPrice', 'lastPrice',
       'priceChange', 'bidPrice', 'askPrice', 'volume', 'openInterest',
       'premium', 'tradeTime', 'longSymbol', 'optionType', 'symbol'],
      dtype='object')
Calls DataFrame Sample:
   strike openPrice highPrice lowPrice  lastPrice priceChange bidPrice  \
6   25.00       N/A     46.03    46.03      46.03       -1.67      N/A   
57  51.00       N/A     20.03    20.03      20.03       -1.67      N/A   
60  52.00       N/A     19.03    19.03      19.03       -1.67      N/A   
62  52.50       N/A     18.53    18.53      18.53       -1.67      N/A   
68  55.00       N/A     16.03    16.03      16.03       -1.67      N/A   

   askPrice  volume  openInterest    premium tradeTime longSymbol optionType  \
6       N/A       0             0  46,030.00  02/05/25  CLH5|250C       Call   
57      N/A    

In [13]:
def get_option_data(results, symbol, strike, option_type):
    """
    Retrieves the row of options data for a given symbol, strike price, and option type.

    Parameters:
    - results (dict): Dictionary containing options data.
    - symbol (str): The symbol to filter.
    - strike (float): The strike price to filter.
    - option_type (str): Either 'call' or 'put'.

    Returns:
    - DataFrame: The filtered row(s) if found, otherwise None.
    """
    if symbol not in results:
        print(f"Symbol '{symbol}' not found.")
        return None

    calls_df, puts_df = results[symbol]

    # Select the appropriate DataFrame
    df = calls_df if option_type.lower() == "call" else puts_df if option_type.lower() == "put" else None
    
    if df is None:
        print("Invalid option type. Choose 'call' or 'put'.")
        return None

    if "strike" not in df.columns:
        print("Strike column not found in the DataFrame.")
        return None

    # Filter by strike price
    filtered_row = df[df["strike"] == strike]

    if filtered_row.empty:
        print(f"No data found for {symbol} {option_type.upper()} with Strike: {strike}")
        return None

    return filtered_row

# Example usage:
result = get_option_data(results, "CLK25", "64.00", "put")  
if result is not None:
    print(result)


    strike openPrice highPrice lowPrice  lastPrice priceChange bidPrice  \
101  64.00      1.20      1.30     1.20        1.3       +0.23     1.22   

    askPrice  volume  openInterest   premium tradeTime longSymbol optionType  \
101     1.25     292             0  1,300.00  02/05/25  CLK5|640P        Put   

        symbol  
101  CLK5|640P  


In [14]:
import numpy as np
import pandas as pd
from scipy.optimize import brentq
from scipy.stats import norm

# Black model price function
def black_model_price(F, K, T, r, sigma, option_type):
    F = float(F)
    K = float(K)
    T = float(T)
    sigma = float(sigma)
    
    d1 = (np.log(F / K) + (0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == "C":  # Call option
        return F * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == "P":  # Put option
        return K * np.exp(-r * T) * norm.cdf(-d2) - F * norm.cdf(-d1)
    else:
        raise ValueError("Invalid option type: must be 'C' or 'P'")


# Function to calculate implied volatility
def implied_volatility(option_price, F, K, T, r, option_type):
    try:
        return brentq(lambda sigma: black_model_price(F, K, T, r, sigma, option_type) - float(option_price), 
                      0.001, 3.0, xtol=1e-6)
    except ValueError:
        return np.nan  # Return NaN if IV cannot be solved

# Assume a constant risk-free rate (adjust as needed)
risk_free_rate = 0.03  # Example: 3%

# Merging futures data with options data
merged_data = []

for symbol, (calls_df, puts_df) in results.items():
    future_row = futures_df[futures_df['Contract'] == symbol]
    if future_row.empty:
        continue  # Skip if no matching future found
    
    F = future_row['Last Price'].values[0]  # Futures price
    T = future_row['Options Days to Expiry'].values[0] / 365.0  # Convert to years
    
    # Process Call Options
    for _, row in calls_df.iterrows():
        K = float(row['strike'])  # Ensure K is a float

        option_price = float(row['lastPrice'])
        iv = implied_volatility(option_price, F, K, T, risk_free_rate, "C")
        merged_data.append((symbol, "Call", K, T, option_price, iv))
    
    # Process Put Options
    for _, row in puts_df.iterrows():
        K = float(row['strike'])  # Ensure K is a float

        option_price = float(row['lastPrice'])
        iv = implied_volatility(option_price, F, K, T, risk_free_rate, "P")
        merged_data.append((symbol, "Put", K, T, option_price, iv))

# Convert to DataFrame
implied_vol_df = pd.DataFrame(merged_data, columns=["Symbol", "Type", "Strike", "Time to Expiry", "Market Price", "Implied Volatility"])

# Display Results
print(implied_vol_df.head())


  Symbol  Type  Strike  Time to Expiry  Market Price  Implied Volatility
0  CLH25  Call    25.0        0.024658         46.03                 NaN
1  CLH25  Call    51.0        0.024658         20.03                 NaN
2  CLH25  Call    52.0        0.024658         19.03                 NaN
3  CLH25  Call    52.5        0.024658         18.53                 NaN
4  CLH25  Call    55.0        0.024658         16.03                 NaN


In [15]:
import plotly.graph_objects as go
import numpy as np

# Loop through unique symbols
for symbol in implied_vol_df['Symbol'].unique():
    # Filter the data for the specific symbol
    symbol_data = implied_vol_df[implied_vol_df['Symbol'] == symbol]
    
    # Get the current futures price (F) for the symbol
    future_row = futures_df[futures_df['Contract'] == symbol]
    F = future_row['Last Price'].values[0]  # Futures price
    print(symbol, F)
    
    # Find the ATM strike (closest to F)
    atm_strike = min(symbol_data['Strike'], key=lambda x: abs(x - F))
    
    # Limit strikes to ±15 from ATM
    symbol_data = symbol_data[symbol_data['Strike'].between(atm_strike - 15, atm_strike + 15)]
    
    # Separate Call and Put data
    call_data = symbol_data[symbol_data['Type'] == 'Call']
    put_data = symbol_data[symbol_data['Type'] == 'Put']
    
    # Create figure
    fig = go.Figure()

    # Plot for Call Options
    fig.add_trace(go.Scatter(x=call_data['Strike'], y=call_data['Implied Volatility'], 
                             mode='markers', name=f'Call Options - {symbol}', 
                             marker=dict(symbol='circle', color='blue', size=8, opacity=0.7)))

    # Plot for Put Options
    fig.add_trace(go.Scatter(x=put_data['Strike'], y=put_data['Implied Volatility'], 
                             mode='markers', name=f'Put Options - {symbol}', 
                             marker=dict(symbol='x', color='red', size=8, opacity=0.7)))

    # Add vertical line for Current Price (F)
    fig.add_trace(go.Scatter(x=[F, F], y=[0, max(symbol_data['Implied Volatility'])], 
                             mode='lines', name=f'Current Price (F) - {symbol}', 
                             line=dict(color='green', dash='dash')))

    # Check for Skew and Smile
    atm_call_iv = call_data[call_data['Strike'] == atm_strike]['Implied Volatility'].values
    atm_put_iv = put_data[put_data['Strike'] == atm_strike]['Implied Volatility'].values

    if len(atm_call_iv) > 0 and len(atm_put_iv) > 0:
        atm_call_iv = atm_call_iv[0]
        atm_put_iv = atm_put_iv[0]

        if atm_put_iv > atm_call_iv:
            skew_type = "Left Skew (Put IV > Call IV)"
        elif atm_put_iv < atm_call_iv:
            skew_type = "Right Skew (Call IV > Put IV)"
        else:
            skew_type = "Balanced Smile"
        
        print(f"{symbol} - {skew_type}")

        # Add Skew Type Annotation to the Plot
        fig.add_annotation(
            x=atm_strike, 
            y=max(symbol_data['Implied Volatility']), 
            text=skew_type, 
            showarrow=False, 
            font=dict(size=12, color="black"),
            align="center",
            bordercolor="black",
            borderwidth=1,
            borderpad=4,
            bgcolor="white",
        )

    # Update layout
    fig.update_layout(
        title=f'Implied Volatility Smile for {symbol}',
        xaxis_title='Strike Price',
        yaxis_title='Implied Volatility',
    )
    
    # Show the plot
    fig.show()


CLH25 71.25
CLH25 - Left Skew (Put IV > Call IV)


CLJ25 70.92
CLJ25 - Left Skew (Put IV > Call IV)


CLK25 70.47
CLK25 - Left Skew (Put IV > Call IV)


CLM25 69.99
CLM25 - Left Skew (Put IV > Call IV)


CLN25 69.54
CLN25 - Left Skew (Put IV > Call IV)


CLQ25 69.02
CLQ25 - Left Skew (Put IV > Call IV)


CLU25 68.4
CLU25 - Left Skew (Put IV > Call IV)


CLV25 68.0
CLV25 - Left Skew (Put IV > Call IV)


CLX25 67.6


CLZ25 67.29
CLZ25 - Left Skew (Put IV > Call IV)


In [16]:
import plotly.express as px

fig = px.line(implied_vol_df, x="Time to Expiry", y="Implied Volatility", color="Symbol",
              markers=True, title="Implied Volatility Term Structure")

fig.show()


In [17]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd

# Initialize lists to store data for the surface plot
strike_prices = []
contract_months = []
implied_vols = []

# Loop through unique contract symbols (months)
for symbol in implied_vol_df['Symbol'].unique():
    # Filter the data for the specific symbol
    symbol_data = implied_vol_df[implied_vol_df['Symbol'] == symbol]
    
    # Get the current futures price (F) for the symbol
    future_row = futures_df[futures_df['Contract'] == symbol]
    F = future_row['Last Price'].values[0]  # Futures price
    
    # Find the ATM strike (closest to F)
    atm_strike = min(symbol_data['Strike'], key=lambda x: abs(x - F))
    
    # Limit strikes to ±15 from ATM
    symbol_data = symbol_data[symbol_data['Strike'].between(atm_strike - 10, atm_strike + 10)]
    
    # Append data to lists
    for _, row in symbol_data.iterrows():
        strike_prices.append(row['Strike'])
        contract_months.append(symbol)
        implied_vols.append(row['Implied Volatility'])

# Convert to NumPy arrays for surface plotting
X = np.array(strike_prices)
Y = np.array(contract_months)  # This is categorical; it will be converted to numerical
Z = np.array(implied_vols)

# Create the surface plot
fig = go.Figure(data=[go.Mesh3d(
    x=X,
    y=pd.factorize(Y)[0],  # Convert categorical contract months to numerical indices
    z=Z,
    colorbar_title='Implied Volatility',
    colorscale='Viridis',
    opacity=0.8
)])

# Update layout for better visibility
fig.update_layout(
    title='Implied Volatility Surface',
    width=1000, height=700,  # Increase figure size
    scene=dict(
        xaxis_title='Strike Price',
        yaxis_title='Contract Month',
        zaxis_title='Implied Volatility',
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.0))  # Adjust viewing angle
    )
)

# Show the plot
fig.show()


In [18]:
import numpy as np
import pandas as pd
from scipy.stats import norm

# Function to adjust implied volatility for Asian options
def adjust_iv_for_asian(iv, n):
    """Adjusts the implied volatility for an Asian option using averaging adjustment."""
    return iv * np.sqrt(2 / (n + 1))

# Asian option pricing (Turnbull-Wakeman approximation for arithmetic average price options)
def asian_option_price(F, K, T, r, sigma_asian, option_type):
    """Computes the price of an Asian option using Turnbull-Wakeman approximation."""
    sigma_adj = sigma_asian * np.sqrt(3 / (2 * (T + 1/365)))  # Adjusted volatility
    d1 = (np.log(F / K) + (0.5 * sigma_adj ** 2) * T) / (sigma_adj * np.sqrt(T))
    d2 = d1 - sigma_adj * np.sqrt(T)
    
    if option_type == "Call":  # Call option
        return np.exp(-r * T) * (F * norm.cdf(d1) - K * norm.cdf(d2))
    elif option_type == "Put":  # Put option
        return np.exp(-r * T) * (K * norm.cdf(-d2) - F * norm.cdf(-d1))
    else:
        raise ValueError("Invalid option type: must be 'C' or 'P'")

# Assume an averaging period (e.g., 30 days for a monthly Asian option)
n = 30  # Number of averaging periods
risk_free_rate = 0.03  # Example risk-free rate (3%)


# Creating a new DataFrame for Asian option calculations
asian_option_data = []

for _, row in implied_vol_df.iterrows():
    symbol = row["Symbol"]
    option_type = row["Type"]
    K = row["Strike"]
    T = row["Time to Expiry"]
    iv = row["Implied Volatility"]
    
    # Find the corresponding futures price for the option's underlying symbol
    future_row = futures_df[futures_df['Contract'] == symbol]
    if future_row.empty:
        continue  # Skip if no matching future found
    
    F = future_row['Last Price'].values[0]  # Assign actual futures price
    
    # Adjust the implied volatility for Asian options
    sigma_asian = adjust_iv_for_asian(iv, n)
    
    # Calculate the Asian option price
    asian_price = asian_option_price(F, K, T, risk_free_rate, sigma_asian, option_type)
    
    # Store results
    asian_option_data.append((symbol, option_type, K, T, sigma_asian, asian_price))


# Convert to DataFrame
asian_option_df = pd.DataFrame(asian_option_data, columns=[
    "Symbol", "Type", "Strike", "Time to Expiry", "Adjusted Asian IV", "Asian Option Price"
])

# Display Results
print("=== Implied Volatility from American Options ===")
print(implied_vol_df)
print("\n=== Asian Option Adjusted IV & Price ===")
print(asian_option_df)


=== Implied Volatility from American Options ===
     Symbol  Type  Strike  Time to Expiry  Market Price  Implied Volatility
0     CLH25  Call    25.0        0.024658         46.03                 NaN
1     CLH25  Call    51.0        0.024658         20.03                 NaN
2     CLH25  Call    52.0        0.024658         19.03                 NaN
3     CLH25  Call    52.5        0.024658         18.53                 NaN
4     CLH25  Call    55.0        0.024658         16.03                 NaN
...     ...   ...     ...             ...           ...                 ...
2192  CLZ25   Put    92.5        0.780822         25.60            0.370879
2193  CLZ25   Put    93.0        0.780822         26.07            0.373706
2194  CLZ25   Put    95.0        0.780822         27.99            0.386817
2195  CLZ25   Put   100.0        0.780822         32.85            0.420562
2196  CLZ25   Put   105.0        0.780822         37.79            0.456233

[2197 rows x 6 columns]

=== Asian Opt

In [20]:
def get_asian_option_data(asian_option_df, symbol, strike, option_type):
    """
    Retrieves the row of Asian options data for a given symbol, strike price, and option type.

    Parameters:
    - asian_option_df (pd.DataFrame): DataFrame containing Asian options data.
    - symbol (str): The symbol to filter.
    - strike (float): The strike price to filter.
    - option_type (str): Either 'Call' or 'Put'.

    Returns:
    - DataFrame: The filtered row(s) if found, otherwise None.
    """
    # Filter based on symbol, strike, and option type
    filtered_row = asian_option_df[
        (asian_option_df["Symbol"] == symbol) &
        (asian_option_df["Strike"] == strike) &
        (asian_option_df["Type"] == option_type)
    ]

    if filtered_row.empty:
        print(f"No Asian option data found for {symbol} {option_type} with Strike: {strike}")
        return None

    return filtered_row

# Example usage:
result = get_asian_option_data(asian_option_df, "CLK25", 77.00, "Put")
if result is not None:
    print(result)


    Symbol Type  Strike  Time to Expiry  Adjusted Asian IV  Asian Option Price
888  CLK25  Put    77.0        0.191781           0.079765            2.961494
